In [ ]:
import subprocess
import shlex
import json
import re

import numpy as np

from pathlib import Path
from IPython.display import display, Markdown

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?<![A-Z])(?=[A-Z])")

In [ ]:
target_directory = json.loads(subprocess.run("cargo metadata --format-version 1".split(), capture_output=True).stdout)["target_directory"]

In [ ]:
def simulate_cpu_reporting_dynamic(
    algorithm, scenario, speciation=0.001, seed=42, sample=1.0, output="profile",
):
    config = "".join(f"""
    (
        speciation: {speciation},
        sample: {sample},
        seed: {seed},

        algorithm: {algorithm},

        scenario: {scenario},

        reporters: [
            Plugin(
                library: "{target_directory}/release/deps/libnecsim_plugins_common.so",
                reporters: [Counter(), Execution()]
            )
        ],
    )
    """.split()).replace(",)", ")").replace(",]", "]")
    
    # Profile the full simulation using perf, then filter the events and output for the Firefox Profiler
    print(
        f"perf record -F {997} --call-graph dwarf -g {target_directory}/release/rustcoalescence simulate '{config}'" +
        f" && perf script -F +pid > {output}.perf && sleep {5}"
    )

In [ ]:
subprocess.run(shlex.split(
    f"cargo build --manifest-path {target_directory}/../rustcoalescence/Cargo.toml --release "
    + "--features rustcoalescence-algorithms-monolithic,rustcoalescence-algorithms-independent,"
    + "rustcoalescence-algorithms-cuda"
), check=True, capture_output=True, text=True);

In [ ]:
display(Markdown("# RAM Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("free"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# CPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("lscpu"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# GPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("nvidia-smi"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

In [ ]:
seed = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64")

for algorithm in [
    "Classical()", "Gillespie()", "SkippingGillespie()",
    f"""Independent(
        dedup_cache: Relative(factor: {1.0}),
        delta_t: {2.0},
        parallelism_mode: Monolithic(event_slice: {100*100*100})
    )""",
    f"""CUDA(
        ptx_jit: true,
        dedup_cache: Relative(factor: {1.0}),
        delta_t: {2.0},
        parallelism_mode: Monolithic(event_slice: {100*100*100})
    )"""
]:
    for scenario, sample in [
        (f"NonSpatial(area: ({100}, {100}), deme: {100})", 1.0),
        (f"AlmostInfinite(radius: {564}, sigma: {10.0})", 1.0),
        (f"""SpatiallyExplicit(
            habitat: "{target_directory}/../maps/madingley/fg0size12/habitat.tif",
            dispersal: "{target_directory}/../maps/madingley/fg0size12/dispersal.tif"
        )""", 0.000025),
    ]:
        simulate_cpu_reporting_dynamic(
            algorithm, scenario, speciation=0.001, sample=sample, seed=seed,
            output=(
                CC_PATTERN.sub('-', algorithm[:algorithm.find('(')]).lower() + "-" +
                CC_PATTERN.sub('-', scenario[:scenario.find('(')]).lower()
            ),
        )